In [ ]:
# Imports
# type: ignore
from __future__ import annotations
from abc import ABC, abstractmethod
from decimal import Decimal
from pprint import pprint
from typing_extensions import override
from typing import *
import unittest

import aiohttp

from vindicator import (
    WynnApi,
    WynnDataRepository
)
from vindicator.constants import *
from vindicator.typehints import *
logger.success('Finished imports')

In [ ]:
# Global SetUp
api = WynnApi()
logger.success('Finished Global SetUp')

In [ ]:
functions = ['className', 'getIndent', 'printDictTypes', 'get_full_player_stats', 'get_guild_stats', 'get_online_players']

def className(obj: Any) -> str:
    return obj.__class__.__name__

def getIndent(depth: int) -> str:
    return '    ' * depth

def printDictTypes(dict_: Dict[str, Any], depth: int = 0) -> None:
    for key, value in dict_.items():
        if isinstance(value, dict):
            print(f'{getIndent(depth)}{key}: dict')
            printDictTypes(value, depth + 1)
        else:
            print(f'{getIndent(depth)}{key}: {className(value)}')

async def get_full_player_stats(username_or_uuid: str) -> Tuple[Dict[str, Any], Dict[str, Any]]:
    async with aiohttp.ClientSession() as session:
        async with session.get(f"https://api.wynncraft.com/v3/player/{username_or_uuid}?fullResult") as resp:
            return (await resp.json(), resp.headers)

async def get_guild_stats(guildname_or_prefix: str, is_prefix: bool = False) -> Tuple[Dict[str, Any], Dict[str, Any]]:
    async with aiohttp.ClientSession() as session:
        async with session.get(f"https://api.wynncraft.com/v3/guild/{'prefix/' if is_prefix else ''}{guildname_or_prefix}") as resp:
            return (await resp.json(), resp.headers)

async def get_online_players() -> Tuple[Dict[str, Any], Dict[str, Any]]:
    async with aiohttp.ClientSession() as session:
        async with session.get("https://api.wynncraft.com/v3/player?identifier=uuid") as resp:
            return (await resp.json(), resp.headers)
logger.success('Finished defining functions')

# Sandbox

In [ ]:
from __future__ import annotations
ID = TypeVar('ID', contravariant=True)
T = TypeVar('T')
S = TypeVar('S')
U = TypeVar('U', bound=type)

def interface(t: U) -> U: return t
def abstract(t: U) -> U: return t

@interface
class Table(Generic[T, ID], Protocol):
    def create(self, entity: T) -> bool: ...
    def exists(self, id: ID) -> bool: ...
    def count(self) -> float: ...
    def findOne(self, id: ID) -> None | T: ...
    def findAll(self) -> None | list[T]: ...
    def update(self, entity: T) -> bool: ...
    def delete(self, id: ID) -> bool: ...

@abstract
class MariaRepository(ABC, Generic[S]):
    def __init__(self, db: Any, logger: Any) -> None:
        self._db: Any = db
        self._logger: Any = logger
    def _prepareStatement(self, connection: Any, sql: str, objects: list[object]) -> Any: ...
    def _execute(self, sql: str, strings: list[object]) -> bool: ...
    # def _execute(self, connection: Any, sql: str, objects: list[object]) -> bool: ...
    def _executeQuery(self, sql: str, objects: list[object]) -> None | Any: ...
    def _logResponseException(self, e: Any) -> None: ...
    def _bindAll(self, res: Any) -> list[S]: ...
    @abstractmethod
    def _bind(self, res: Any) -> S: ...

@interface
class GuildId(Protocol):
    def getName(self) -> str: ...

class Guild(GuildId):
    def __init__(self, name: str, prefix: str, createdAt: dt) -> None:
        self._name: str = name
        self._prefix: str = prefix
        self._createdAt: dt = createdAt
    @override
    def getName(self) -> str: return self._name
    def getPrefix(self) -> str: return self._prefix
    def getCreatedAt(self) -> dt: return self._createdAt

@interface
class CharacterHistoryBase(Table[Guild, GuildId], Protocol):
    def findAllIn(self, guildNames: list[str]) -> None | list[Guild]: ...
    def findAllCaseInsensitive(self, guildName: str) -> None | list[Guild]: ...
    def findAllByPrefix(self, prefix: str) -> None | list[Guild]: ...
    def findAllByPrefixCaseInsensitive(self, prefix: str) -> None | list[Guild]: ...

class CharacterHistoryTable(MariaRepository[Guild], CharacterHistoryBase):
    """class::MariaGuildRepository"""
    _DB_FORMAT: Any
    def __init__(self, db: Any, logger: Any) -> None:
        super().__init__(db, logger)
    @override
    def _bind(self, res: Any) -> Guild: ...
    @override
    def create(self, entity: Guild) -> bool: ...
    @override
    def exists(self, id: GuildId) -> bool: ...
    @override
    def count(self) -> float: ...
    @override
    def findOne(self, id: GuildId) -> None | Guild: ...
    @override
    def findAllIn(self, guildNames: list[str]) -> None | list[Guild]: ...
    def findAllCaseInsensitive(self, guildName: str) -> None | list[Guild]: ...
    def findAllByPrefix(self, prefix: str) -> None | list[Guild]: ...
    def findAllByPrefixCaseInsensitive(self, prefix: str) -> None | list[Guild]: ...
    @override
    def findAll(self) -> None | list[Guild]: ...
    @override
    def update(self, entity: Guild) -> bool: ...
    @override
    def delete(self, id: GuildId) -> bool: ...


In [2]:
class Foo:
    def __init__(self, dict: dict[str, dict[str, int]]) -> None:
        self.value: dict[str, int] = dict.get("value", {})

foo1 = Foo({"value": {"foo": 1}})
print(foo1.value)
foo2 = Foo({})
foo2.value["test"] = 1
print(foo2.value)
foo3 = Foo({})
print(foo3.value)
foo2.value["test2"] = 2
print(foo2.value)
print(foo3.value)

{'foo': 1}
{'test': 1}
{}
{'test': 1, 'test2': 2}
{}
